In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

Instructions for updating:
Use the retry module or similar alternatives.


In [3]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

In [10]:
# Step 1: Read in data
#using TF Learn's built in function to load MNIST data to the folder data/mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_DATA', one_hot = True)

Extracting MNIST_DATA\train-images-idx3-ubyte.gz
Extracting MNIST_DATA\train-labels-idx1-ubyte.gz
Extracting MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA\t10k-labels-idx1-ubyte.gz


In [5]:
# Step 2: Create placeholders for features and labels
#each image in the MNIST data is of shape 28*28 = 784
#therefore, each image is represented with a 1*784 tensor
#there are 10 classes for each image, corresponding to digits 0 - 9
# each lable is one hot vector

X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
Y= tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')

In [6]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y

w = tf.Variable(tf.random_normal(shape=[784, 10], stddev= 0.01), name='weights')
tf.summary.histogram("weights",w) #可视化观看变量

b = tf.Variable(tf.zeros([1, 10]), name="bias")
tf.summary.histogram("biases",b) #可视化观看变量

<tf.Tensor 'biases:0' shape=() dtype=string>

In [7]:
# Step 4: build model
# the model that returns the logits
# this logits will be later passed through softmax layer

y_predicted = tf.matmul(X, w) + b

In [8]:
# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function

entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits = y_predicted, name='loss')
# computes the mean over all the examples in the batch
loss = tf.reduce_mean(entropy)
tf.summary.histogram("loss",loss) #可视化观看变量

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



<tf.Tensor 'loss_1:0' shape=() dtype=string>

In [9]:
# Step 6: define training op
#using gradient descent with learning rate of 0.01 to minimize loss

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    # to visualize using TensorBoard
    
    merged = tf.summary.merge_all()  #合并到Summary中    
    writer = tf.summary.FileWriter('./graphs/logistic_reg', sess.graph)
    
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    # train the model n_epochs times
    for i in range(n_epochs):
        total_loss = 0
        
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch})
            total_loss += loss_batch
        print ('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
    print ('Total time: {0} seconds'.format(time.time() - start_time))
    
    #should be around 0.35 after 25 epochs
    print('Optimization Finished!')
    
    #test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, y_predicted], feed_dict={X: X_batch, Y: Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)
        
    print ('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

    writer.close()


Average loss epoch 0: 1.2869023893262956
Average loss epoch 1: 0.7326213213113638
Average loss epoch 2: 0.6006098071456233
Average loss epoch 3: 0.5366255961931669
Average loss epoch 4: 0.49805550503008295
Average loss epoch 5: 0.4714476910782305
Average loss epoch 6: 0.45080521438783144
Average loss epoch 7: 0.436402122388075
Average loss epoch 8: 0.4242247862693591
Average loss epoch 9: 0.4124554686057262
Average loss epoch 10: 0.4043899706124148
Average loss epoch 11: 0.39680068535265667
Average loss epoch 12: 0.39000507060325507
Average loss epoch 13: 0.3843660047560027
Average loss epoch 14: 0.3789723893770805
Average loss epoch 15: 0.3752240930403863
Average loss epoch 16: 0.3705409522795733
Average loss epoch 17: 0.36530268612580424
Average loss epoch 18: 0.3639358791646424
Average loss epoch 19: 0.358975140330119
Average loss epoch 20: 0.35614274921533945
Average loss epoch 21: 0.35384415465218205
Average loss epoch 22: 0.35291586719053886
Average loss epoch 23: 0.3477450754745

<img src="./graphs/logistic_reg/graph_large_attrs_key=_too_large_attrs&limit_attr_size=1024&run= (1).png" >